# IERG4160 2023-24 Spring: Individual Project - Neural network training

## Section 0 Environment Setup

**Importing Libraries**

In [ ]:
import math
import statistics
import random
import string
import timeit
from datetime import datetime
from typing import List, Optional, Tuple
import matplotlib.pyplot as plt
import unicodedata
import os
import csv
import cv2
from PIL import Image
import pandas as pd
import numpy as np
import copy
import torch
from torch.utils.data import random_split, Dataset, DataLoader, ConcatDataset, Subset
from torch.optim.optimizer import (Optimizer, required, _use_grad_for_differentiable, _default_to_fused_or_foreach,
                        _differentiable_doc, _foreach_doc, _maximize_doc)
import torchvision.transforms as transforms

**Utilize GPU**

In [ ]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device = get_device()

**Download MINST Dataset**

In [ ]:
from torchvision.datasets import MNIST

# Download Dataset
MNIST_train_dataset = MNIST(root='./Datasets', train=True, download=True, transform=transforms.ToTensor())
MNIST_test_dataset = MNIST(root='./Datasets', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
# Visualize the raw data from the dataset
print("=== Raw Data Samples from the MNIST Train Dataset ===")
for i in range(3):
    image, label = MNIST_train_dataset[i]
    image = image.squeeze().numpy()
    plt.imshow(image, cmap='gray')
    plt.title(f"Label: {label}")
    plt.show()

print("=== Raw Data Samples from the MNIST Test Dataset ===")
for i in range(3):
    image, label = MNIST_test_dataset[i]
    image = image.squeeze().numpy()
    plt.imshow(image, cmap='gray')
    plt.title(f"Label: {label}")
    plt.show()

**Helping Functions**

In [ ]:
global train_start_time

def load_image(filename):
  im_pil = Image.open(filename)
  im = np.array(im_pil).astype(np.float32) / 255
  return im

def convert_to_list(input_list):
    if not isinstance(input_list, list):
        input_list = [input_list]
    return input_list

def plot_time_history(time_history_list=[], save=True, x_label_name="Epochs", y_label_name="Culminative Time Used", title_name="Time History"):
    time_history_list = convert_to_list(time_history_list)
    for i, time_history in enumerate(time_history_list):
        plt.plot(time_history, label=f"Time History {i+1}")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    if len(time_history_list) > 1:
        plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_time_history_{train_start_time}.png')
    plt.show()

def plot_loss_history(train_loss_history_list=[], test_loss_history_list=[], save=True, x_label_name="Epochs", y_label_name="Loss", title_name="Loss History"):
    train_loss_history_list = convert_to_list(train_loss_history_list)
    test_loss_history_list = convert_to_list(test_loss_history_list)
    for i, train_loss_history in enumerate(train_loss_history_list):
        plt.plot(train_loss_history, label=f"Train Loss History {i+1}")
    for i, test_loss_history in enumerate(test_loss_history_list):
        plt.plot(test_loss_history, label=f"Test Loss History {i+1}")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    if (len(train_loss_history_list) + len(test_loss_history_list)) > 1:
        plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_loss_history_{train_start_time}.png')
    plt.show()

def plot_accuracy_history(train_accuracy_history_list=[], test_accuracy_history_list=[], save=True, x_label_name="Epochs", y_label_name="Accuracy", title_name="Accuracy History"):
    train_accuracy_history_list = convert_to_list(train_accuracy_history_list)
    test_accuracy_history_list = convert_to_list(test_accuracy_history_list)
    for i, train_accuracy_history in enumerate(train_accuracy_history_list):
        plt.plot(train_accuracy_history, label=f"Train Accuracy History {i+1}")
    for i, test_accuracy_history in enumerate(test_accuracy_history_list):
        plt.plot(test_accuracy_history, label=f"Test Accuracy History {i+1}")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    if (len(train_accuracy_history_list) + len(test_accuracy_history_list)) > 1:
        plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_accuracy_history_{train_start_time}.png')
    plt.show()

def plot_error_history(train_error_history_list=[], test_error_history_list=[], save=True, x_label_name="Epochs", y_label_name="Error", title_name="Error History"):
    train_error_history_list = convert_to_list(train_error_history_list)
    test_error_history_list = convert_to_list(test_error_history_list)
    for i, train_error_history in enumerate(train_error_history_list):
        plt.plot(train_error_history, label=f"Train Error History {i+1}")
    for i, test_error_history in enumerate(test_error_history_list):
        plt.plot(test_error_history, label=f"Test Error History {i+1}")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    if (len(train_error_history_list) + len(test_error_history_list)) > 1:
        plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_error_history_{train_start_time}.png')
    plt.show()

def plot_time_history_single(time_history, save=True, x_label_name="Epochs", y_label_name="Culminative Time Used", title_name="Time History"):
    plt.plot(time_history, label=f"Time History")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_time_history_single_{train_start_time}.png')
    plt.show()

def plot_loss_history_single(train_loss_history, test_loss_history, save=True, x_label_name="Epochs", y_label_name="Loss", title_name="Loss History"):
    plt.plot(train_loss_history, label=f"Train Loss History")
    plt.plot(test_loss_history, label=f"Test Loss History")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_loss_history_single_{train_start_time}.png')
    plt.show()

def plot_accuracy_history_single(train_accuracy_history, test_accuracy_history, save=True, x_label_name="Epochs", y_label_name="Accuracy", title_name="Accuracy History"):
    plt.plot(train_accuracy_history, label=f"Train Accuracy History")
    plt.plot(test_accuracy_history, label=f"Test Accuracy History")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_accuracy_history_single_{train_start_time}.png')
    plt.show()

def plot_error_history_single(train_error_history, test_error_history, save=True, x_label_name="Epochs", y_label_name="Error", title_name="Error History"):
    plt.plot(train_error_history, label=f"Train Error History")
    plt.plot(test_error_history, label=f"Test Error History")
    plt.xlabel(x_label_name)
    plt.ylabel(y_label_name)
    plt.title(title_name)
    plt.legend()
    if save:
        plt.savefig(f'{current_dataset_name}_error_history_single_{train_start_time}.png')
    plt.show()

def get_accuracy(outputs, labels):
    with torch.no_grad():
        if outputs.dim() > 1:
            _, predictions = torch.max(outputs, dim=1)
        else:
            predictions = outputs
        return torch.tensor(torch.sum(predictions == labels).item() / len(predictions))

def get_error(outputs, labels):
    with torch.no_grad():
        if outputs.dim() > 1:
            _, predictions = torch.max(outputs, dim=1)
        else:
            predictions = outputs
        return torch.tensor(torch.sum(predictions != labels).item() / len(predictions))

## Section 1 Task 1 Neural Network Parameters

**Iterate Algorithms**

In [ ]:
def evaluate_model_simple(model, dataloader, loss_func=torch.nn.functional.cross_entropy, accuracy_func=get_accuracy, error_func=get_error):
    losses = []
    accuracies = []
    errors = []
    with torch.no_grad():
        for features, labels in dataloader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = loss_func(outputs, labels)
            accuracy = accuracy_func(outputs, labels)
            error = error_func(outputs, labels)
            losses.append(loss)
            accuracies.append(accuracy)
            errors.append(error)
        loss_average = torch.stack(losses).mean().item()
        accuracy_average = torch.stack(accuracies).mean().item()
        error_average = torch.stack(errors).mean().item()
    return loss_average, accuracy_average, error_average

def iterate_model_simple(model, dataloader, num_epochs, optimizer, loss_func=torch.nn.functional.cross_entropy, accuracy_func=get_accuracy, error_func=get_error, show_history=True, test_dataloader=None, include_intial_history=False):
    loss_history = []
    accuracy_history = []
    error_history = []
    time_history = []
    start_time = timeit.default_timer()

    test_loss_history = []
    test_accuracy_history = []
    test_error_history = []

    if include_intial_history is True:
        loss, accuracy, error = evaluate_model_simple(model=model, dataloader=dataloader, loss_func=loss_func, accuracy_func=accuracy_func, error_func=error_func)
        loss_history.append(loss)
        accuracy_history.append(accuracy)
        error_history.append(error)
        if test_dataloader is not None:
            test_loss, test_accuracy, test_error = evaluate_model_simple(model=model, dataloader=test_dataloader, loss_func=loss_func, accuracy_func=accuracy_func, error_func=error_func)
            test_loss_history.append(test_loss)
            test_accuracy_history.append(test_accuracy)
            test_error_history.append(test_error)

    model.train()
    for epoch in range(num_epochs):
        losses = []
        accuracies = []
        errors = []
        for batch, (features, labels) in enumerate(dataloader):
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = loss_func(outputs, labels)
            accuracy = accuracy_func(outputs, labels)
            error = error_func(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss.detach()
            losses.append(loss)
            accuracies.append(accuracy)
            errors.append(error)
        time_used = timeit.default_timer() - start_time
        time_history.append(time_used)
        loss_average = torch.stack(losses).mean().item()
        accuracy_average = torch.stack(accuracies).mean().item()
        error_average = torch.stack(errors).mean().item()
        loss_history.append(loss_average)
        accuracy_history.append(accuracy_average)
        error_history.append(error_average)
        if test_dataloader is not None:
            test_loss, test_accuracy, test_error = evaluate_model_simple(model=model, dataloader=test_dataloader, loss_func=loss_func, accuracy_func=accuracy_func, error_func=error_func)
            test_loss_history.append(test_loss)
            test_accuracy_history.append(test_accuracy)
            test_error_history.append(test_error)
        if show_history:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss_average:.16f}, Train Accuracy: {accuracy_average:.16f}, Train Error: {error_average:.16f}, Culminative Time Used: {time_used}')
            if test_dataloader is not None:
                print(f'Test Loss: {test_loss:.16f}, Test Accuracy: {test_accuracy:.16f}, Test Error: {test_error:.16f}')
    if test_dataloader is not None:
        return loss_history, accuracy_history, error_history, time_history, test_loss_history, test_accuracy_history, test_error_history
    return loss_history, accuracy_history, error_history, time_history

def train_neural_network_model(model, train_dataloader, test_dataloader, num_epochs, optimizer, loss_func=torch.nn.functional.cross_entropy, accuracy_func=get_accuracy, error_func=get_error, show_history=True, save_result=True, save_path_str="MNIST_CNN", save_file_extra_information=""):
    if test_dataloader is not None:
        train_loss_history, train_accuracy_history, train_error_history, time_history, test_loss_history, test_accuracy_history, test_error_history = iterate_model_simple(model, train_dataloader, num_epochs, optimizer, loss_func, accuracy_func, error_func, show_history, test_dataloader, True)
    else:
        train_loss_history, train_accuracy_history, train_error_history, time_history = iterate_model_simple(model, train_dataloader, num_epochs, optimizer, loss_func, accuracy_func, error_func, show_history)

    # Print learned parameters
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f'{name}: {param.data}')
    
    # Save Results
    if save_result:
        filename = "{}_{}_{}.npy".format(save_path_str, train_start_time, experiment_id)
        with open(filename, 'wb') as f:
            np.savez(f, time_history=time_history, train_loss_history=train_loss_history, train_accuracy_history=train_accuracy_history, train_error_history=train_error_history, test_loss_history=test_loss_history, test_accuracy_history=test_accuracy_history, test_error_history=test_error_history, extra_information=save_file_extra_information)
        torch.save(model.state_dict(), filename + "_model_state_dict.pth")
        logname = "{}_{}_{}.txt".format(save_path_str, train_start_time, experiment_id)
        with open(logname, 'wb') as f:
            f.write(save_file_extra_information.encode('utf-8'))

    # Graph
    if show_history:
        plot_time_history([time_history])
        plot_loss_history([train_loss_history], [test_loss_history])
        plot_accuracy_history([train_accuracy_history], [test_accuracy_history])
        plot_error_history([train_error_history], [test_error_history])
    
    return time_history, train_loss_history, train_accuracy_history, train_error_history, test_loss_history, test_accuracy_history, test_error_history

def experiment_neural_network_model(train_dataset, test_dataset, modelClass, optimizerClass, train_func, epochs_list, learning_rate_list, batch_size_list, loss_func_list, accuracy_func_list, error_func_list, experiment_rounds = 1, show_history=True, save_result=True):
    global experiment_id
    experiment_id = 0

    epochs_list = convert_to_list(epochs_list)
    learning_rate_list = convert_to_list(learning_rate_list)
    batch_size_list = convert_to_list(batch_size_list)
    loss_func_list = convert_to_list(loss_func_list)
    accuracy_func_list = convert_to_list(accuracy_func_list)
    error_func_list = convert_to_list(error_func_list)

    time_history_total = []
    train_loss_history_total = []
    train_accuracy_history_total = []
    train_error_history_total = []
    test_loss_history_total = []
    test_accuracy_history_total = []
    test_error_history_total = []
    
    for n in range(experiment_rounds):
        experiment_id = experiment_id + 1
        print(f'=== Training Experiment {experiment_id} ===')
        print(f'number of epochs is {epochs_list[min(n, len(epochs_list) - 1)]}')
        num_epochs = epochs_list[min(n, len(epochs_list) - 1)]
        print(f'learning rate is {learning_rate_list[min(n, len(learning_rate_list) - 1)]}')
        learning_rate = learning_rate_list[min(n, len(learning_rate_list) - 1)]
        print(f'batch size is {batch_size_list[min(n, len(batch_size_list) - 1)]}')
        batch_size = batch_size_list[min(n, len(batch_size_list) - 1)]
        print(f'loss function is {loss_func_list[min(n, len(loss_func_list) - 1)]}')
        loss_func = loss_func_list[min(n, len(loss_func_list) - 1)]
        print(f'accuracy function is {accuracy_func_list[min(n, len(accuracy_func_list) - 1)]}')
        accuracy_func = accuracy_func_list[min(n, len(accuracy_func_list) - 1)]
        print(f'error function is {error_func_list[min(n, len(error_func_list) - 1)]}')
        error_func = error_func_list[min(n, len(error_func_list) - 1)]
        
        model = modelClass().to(device)

        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        optimizer = optimizerClass(model.parameters(), learning_rate)

        save_path_str = f"IERG4160_Project_{current_dataset_name}_E_{num_epochs}_lr_{learning_rate}_B_{batch_size}"

        global save_file_extra_information
        save_file_extra_information = f"""
        === {save_path_str} ===
        IERG4160 Project Training
        The experiment ID is: {experiment_id}
        The train start time is: {train_start_time}
        The dataset is: {current_dataset_name}

        experiment_rounds = {experiment_rounds}

        modelType = {modelClass.__name__}
        optimizerType = {optimizerClass.__name__}

        num_epochs_list = {epochs_list}
        learning_rate_list = {learning_rate_list}
        batch_size_list = {batch_size_list}
        loss_func_list = {loss_func_list}
        accuracy_func_list = {accuracy_func_list}
        error_func_list = {error_func_list}

        !-- Current Status --!
        num_epochs = {num_epochs}
        learning_rate = {learning_rate}
        batch_size = {batch_size}
        loss_func = {loss_func}
        accuracy_func = {accuracy_func}
        error_func = {error_func}
        """

        time_history, train_loss_history, train_accuracy_history, train_error_history, test_loss_history, test_accuracy_history, test_error_history = train_func(model, train_dataloader, test_dataloader, num_epochs, optimizer, loss_func, accuracy_func, error_func, show_history, save_result, save_path_str, save_file_extra_information)

        time_history_total.append(time_history)
        train_loss_history_total.append(train_loss_history)
        train_accuracy_history_total.append(train_accuracy_history)
        train_error_history_total.append(train_error_history)
        test_loss_history_total.append(test_loss_history)
        test_accuracy_history_total.append(test_accuracy_history)
        test_error_history_total.append(test_error_history)
    
    print(f'=== The Experiment Result ===')
    print(f'Name of current dataset: {current_dataset_name}')
    plot_time_history(time_history_total)
    plot_loss_history(train_loss_history_total, test_loss_history_total)
    plot_accuracy_history(train_accuracy_history_total, test_accuracy_history_total)
    plot_error_history(train_error_history_total, test_error_history_total)


**Experiment**

In [ ]:
# MNIST
# Load Dataset
train_dataset = MNIST_train_dataset
test_dataset = MNIST_test_dataset

# Show Dataset Status
current_dataset_name = "MNIST"
print(f'The current dataset is {current_dataset_name}.')
print(f'Number of samples in the train dataset: {len(train_dataset)}')
print(f'Number of samples in the test dataset: {len(test_dataset)}')

In [ ]:
class MNIST_CNN_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Flatten(),
            torch.nn.Linear(7*7*32, 10)
        )
    
    def forward(self, x):
        x = self.network(x)
        return x

In [ ]:
modelType = MNIST_CNN_Model
optimizerType = torch.optim.SGD
train_func = train_neural_network_model

num_epochs_list = 10
learning_rate_list = 0.03
batch_size_list = 128
loss_func_list = torch.nn.CrossEntropyLoss()
accuracy_func_list = get_accuracy
error_func_list = get_error

experiment_rounds = 1

print(f'The current dataset is {current_dataset_name}.')
train_start_time = datetime.now().strftime("%Y-%m-%d %H.%M.%S")
print(f'The current train start time is {train_start_time}.')

experiment_neural_network_model(train_dataset, test_dataset, modelType, optimizerType, train_func, num_epochs_list, learning_rate_list, batch_size_list, loss_func_list, accuracy_func_list, error_func_list, experiment_rounds)

## Section 2 Task 2 U-Net

**MNIST Noisy Dataset**

Note: Remember in the noisy dataset label is a image, instead of a number in original dataset

In [ ]:
#MNIST Noisy
noisy_sigma = 0.02

def add_gaussian_noise(images, sigma):
    noisy_images = images + sigma * np.random.normal(size = images.shape)
    noisy_images = np.clip(noisy_images, 0.0, 1.0)
    return noisy_images

def convert_pure_image_array(images):
    images = images.astype(np.float32)
    images = np.squeeze(images)
    images = images.reshape(images.shape[0], -1)
    images = (images * 255).astype(np.uint8)
    return images

MNIST_train_dataset_noisy_image = []
MNIST_train_dataset_noisy_label = []
MNIST_test_dataset_noisy_image = []
MNIST_test_dataset_noisy_label = []
for image, _ in MNIST_train_dataset:
    noisy_images = add_gaussian_noise(image.numpy() / 255.0, noisy_sigma)
    noisy_images = convert_pure_image_array(noisy_images)
    truth_images = convert_pure_image_array(image.numpy())
    MNIST_train_dataset_noisy_image.append(noisy_images)
    MNIST_train_dataset_noisy_label.append(truth_images)
for image, _ in MNIST_test_dataset:
    noisy_images = add_gaussian_noise(image.numpy() / 255.0, noisy_sigma)
    noisy_images = convert_pure_image_array(noisy_images)
    truth_images = convert_pure_image_array(image.numpy())
    MNIST_test_dataset_noisy_image.append(noisy_images)
    MNIST_test_dataset_noisy_label.append(truth_images)

In [ ]:
class MNISTNoisy(Dataset):
    def __init__(self, data_sample, targets_sample, train=True, transform=None, target_transform=None):
        self.train = train
        self.data, self.targets = self._load_data(data_sample, targets_sample)
        self.transform = transform
        self.target_transform = target_transform

    def _load_data(self, data_sample, targets_sample):
        data = torch.tensor(data_sample)
        targets = torch.tensor(targets_sample)
        return data, targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]

        img = Image.fromarray(img.numpy(), mode="L")
        target = Image.fromarray(target.numpy(), mode="L")

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

MNIST_train_dataset_noisy = MNISTNoisy(MNIST_train_dataset_noisy_image, MNIST_train_dataset_noisy_label, train=True, transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
MNIST_test_dataset_noisy = MNISTNoisy(MNIST_test_dataset_noisy_image, MNIST_test_dataset_noisy_label, train=False, transform=transforms.ToTensor(), target_transform=transforms.ToTensor())

# Load Dataset
train_dataset = MNIST_train_dataset_noisy
test_dataset = MNIST_test_dataset_noisy

# Show Dataset Status
current_dataset_name = "MNIST Noisy"
print(f'The current dataset is {current_dataset_name}.')
print(f'Number of samples in the train dataset: {len(train_dataset)}')
print(f'Number of samples in the test dataset: {len(test_dataset)}')

In [ ]:
# Visualize the raw data from the dataset
print("=== Raw Data Samples from the MNIST Noisy Train Dataset ===")
for i in range(3):
    image, label = MNIST_train_dataset_noisy[i]
    print("Noisy Image:")
    image = image.numpy().squeeze()
    plt.imshow(image, cmap='gray')
    plt.show()
    print("Ground Truth Image:")
    label = label.numpy().squeeze()
    plt.imshow(label, cmap='gray')
    plt.show()

print("=== Raw Data Samples from the MNIST Noisy Test Dataset ===")
for i in range(3):
    image, label = MNIST_test_dataset_noisy[i]
    print("Noisy Image:")
    image = image.numpy().squeeze()
    plt.imshow(image, cmap='gray')
    plt.show()
    print("Ground Truth Image:")
    label = label.numpy().squeeze()
    plt.imshow(label, cmap='gray')
    plt.show()

**Experiment**

In [ ]:
class MNIST_UNet_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = torch.nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.up1 = torch.nn.ConvTranspose2d(32, 32, kernel_size=2, stride=2)
        self.conv4 = torch.nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.up2 = torch.nn.ConvTranspose2d(32, 32, kernel_size=2, stride=2)
        self.conv5 = torch.nn.Conv2d(48, 32, kernel_size=3, padding=1)
        
    def forward(self, x):
        conv1_out = self.conv1(x)
        pool1_out = self.pool1(conv1_out)
        conv2_out = self.conv2(pool1_out)
        pool2_out = self.pool2(conv2_out)
        conv3_out = self.conv3(pool2_out)
        up1_out = self.up1(conv3_out)
        cat1_out = torch.cat([up1_out, conv2_out], dim=1)
        conv4_out = self.conv4(cat1_out)
        up2_out = self.up2(conv4_out)
        cat2_out = torch.cat([up2_out, conv1_out], dim=1)
        conv5_out = self.conv5(cat2_out)
        return conv5_out
    
class MNIST_UNet_Model_SingleChannel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = torch.nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.up1 = torch.nn.ConvTranspose2d(32, 32, kernel_size=2, stride=2)
        self.conv4 = torch.nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.up2 = torch.nn.ConvTranspose2d(24, 24, kernel_size=2, stride=2)
        self.conv5 = torch.nn.Conv2d(24, 8, kernel_size=3, padding=1)
        self.conv6 = torch.nn.Conv2d(8, 1, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        pool1_out = self.pool1(conv1_out)
        conv2_out = self.conv2(pool1_out)
        pool2_out = self.pool2(conv2_out)
        conv3_out = self.conv3(pool2_out)
        up1_out = self.up1(conv3_out)
        cat1_out = torch.nn.cat([up1_out, conv2_out], dim=1)
        conv4_out = self.conv4(cat1_out)
        up2_out = self.up2(conv4_out)
        cat2_out = torch.nn.cat([up2_out, conv1_out], dim=1)
        conv5_out = self.conv5(cat2_out)
        conv6_out = self.conv6(conv5_out)

        return conv6_out

In [ ]:
modelType = MNIST_UNet_Model
optimizerType = torch.optim.SGD
train_func = train_neural_network_model

# Custom Parameters
custom_image_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((16,16)),
    transforms.ToTensor()
])

custom_target_transform = transforms.Compose([
    transforms.Resize((16,16)),
    transforms.ToTensor()
])

train_dataset.transform = custom_image_transform
train_dataset.target_transform = custom_target_transform
test_dataset.transform = custom_image_transform
test_dataset.target_transform = custom_target_transform
#---------------------#

num_epochs_list = 10
learning_rate_list = 0.03
batch_size_list = 10
loss_func_list = torch.nn.MSELoss()
accuracy_func_list = get_accuracy
error_func_list = get_error

experiment_rounds = 1

print(f'The current dataset is {current_dataset_name}.')
train_start_time = datetime.now().strftime("%Y-%m-%d %H.%M.%S")
print(f'The current train start time is {train_start_time}.')

experiment_neural_network_model(train_dataset, test_dataset, modelType, optimizerType, train_func, num_epochs_list, learning_rate_list, batch_size_list, loss_func_list, accuracy_func_list, error_func_list, experiment_rounds)

## Section 3 Analysis

### Section 3.1 Model Prediction

**Image Prediction**

In [ ]:
# --- Change the parameter manually here!! --- #
predict_image_path = "Test.png"
predict_model_path = "IERG4160_Project_MNIST_E_10_lr_0.03_B_128_2024-03-30 18.36.57_1.npy_model_state_dict.pth"
predict_model_type = MNIST_CNN_Model
predict_image_togrey = True
#----------------------------------------------#

predict_model = predict_model_type()
predict_model.load_state_dict(torch.load(predict_model_path, map_location=device))

if predict_image_togrey is True:
    test_image = 1.0 - cv2.cvtColor(cv2.resize(load_image(predict_image_path), (28, 28)), cv2.COLOR_RGB2GRAY)
else:
    test_image = cv2.resize(load_image(predict_image_path), (28, 28))
plt.imshow(test_image, cmap='gray')

test_image_torch = torch.Tensor(test_image[np.newaxis, np.newaxis])
test_predicted_label_array = predict_model(test_image_torch).detach().numpy()[0, ...]
print('The chances for predicted labels are: ', test_predicted_label_array)

test_predicted_label = np.argmax(test_predicted_label_array)

print('The predicted label is: ', test_predicted_label)

### Section 3.2 Training Performance Visulization

**Clear and Initialize Data**

In [ ]:
data_time_history_total = []
data_train_loss_history_total = []
data_train_accuracy_history_total = []
data_train_error_history_total = []
data_test_loss_history_total = []
data_test_accuracy_history_total = []
data_test_error_history_total = []

# Run this whenever you want to clear all appended data or initially load data!!

**Loading Data Manually**

In [ ]:
# --- Change the parameter manually here!! --- #
filename_load_list = ["IERG4160_Project_MNIST_E_10_lr_0.03_B_128_2024-03-30 18.36.57_1.npy",
                      ""]
data_append_load = True
#----------------------------------------------#

for filename_load in filename_load_list:
    print("============================================")
    print("*** Loading file...                     ***")
    print("============================================")
    try:
        # Load the file
        load_result = np.load(filename_load)
        print('Result has been loaded from the file: ', filename_load)

        # Load the attributes from the file
        data_time_history = load_result['time_history']
        data_train_loss_history = load_result['train_loss_history']
        data_train_accuracy_history = load_result['train_accuracy_history']
        data_train_error_history = load_result['train_error_history']
        data_test_loss_history = load_result['test_loss_history']
        data_test_accuracy_history = load_result['test_accuracy_history']
        data_test_error_history = load_result['test_error_history']

        print("=======Content of the File=======")
        print(load_result.files)

        print("=======STATUS RESULT=======")
        print("Time History: ", data_time_history)

        print("=======TRAIN RESULT=======")
        print("Train Loss History: ", data_train_loss_history)
        print("Train Accuracy History: ", data_train_accuracy_history)
        print("Train Error History: ", data_train_error_history)

        print("=======TEST RESULT=======")
        print("Test Loss History: ", data_test_loss_history)
        print("Test Accuracy History: ", data_test_accuracy_history)
        print("Test Error History: ", data_test_error_history)

        print("=======VISUALIZATION RESULT=======")
        #plot_time_history_single(data_time_history, save=False)
        plot_loss_history_single(data_train_loss_history, data_test_loss_history, save=False)
        plot_accuracy_history_single(data_train_accuracy_history, data_test_accuracy_history, save=False)
        plot_error_history_single(data_train_error_history, data_test_error_history, save=False)

        # Append the data
        if data_append_load:
            data_time_history_total.append(data_time_history)
            data_train_loss_history_total.append(data_train_loss_history)
            data_train_accuracy_history_total.append(data_train_accuracy_history)
            data_train_error_history_total.append(data_train_error_history)
            data_test_loss_history_total.append(data_test_loss_history)
            data_test_accuracy_history_total.append(data_test_accuracy_history)
            data_test_error_history_total.append(data_test_error_history)
    except (FileNotFoundError, IOError):
        print("Failed to load the file: ", filename_load)
    print("")

**Plot for Experiment Parameters**

In [ ]:
# --- Change the parameter manually here!! --- #
plot_parameters_list = [1]
plot_title_strings = ""
plot_legend_strings = ""
plot_save_fig_bool = False
plot_show_train_bool = True
plot_show_test_bool = False
plot_log_scale = False
#----------------------------------------------#

plot_different_parameter_time_history = convert_to_list(data_time_history_total)
for i, plot_time_history in enumerate(plot_different_parameter_time_history):
    plt.plot(plot_time_history, label=f"Time History with {plot_legend_strings} = {plot_parameters_list[i]}")
plt.xlabel("Epochs")
if plot_log_scale is True:
    plt.ylabel("Log Culminative Time Used")
    plt.yscale('log')
else:
    plt.ylabel("Culminative Time Used")
plt.title("Time History" + plot_title_strings)
plt.legend()
if plot_save_fig_bool is True:
    plt.savefig(f'Analysis_{current_dataset_name}_time_history_{plot_legend_strings}_{train_start_time}.png')
plt.show()

plot_different_parameter_train_loss_history = convert_to_list(data_train_loss_history_total)
plot_different_parameter_test_loss_history = convert_to_list(data_test_loss_history_total)
if plot_show_train_bool is True:
    for i, plot_train_loss_history in enumerate(plot_different_parameter_train_loss_history):
        plt.plot(plot_train_loss_history, label=f"Train Loss History with {plot_legend_strings} = {plot_parameters_list[i]}")
if plot_show_test_bool is True:
    for i, plot_test_loss_history in enumerate(plot_different_parameter_test_loss_history):
        plt.plot(plot_test_loss_history, label=f"Test Loss History with {plot_legend_strings} = {plot_parameters_list[i]}")
plt.xlabel("Epochs")
if plot_log_scale is True:
    plt.ylabel("Log Loss")
    plt.yscale('log')
else:
    plt.ylabel("Loss")
plt.title("Loss History" + plot_title_strings)
plt.legend()
if plot_save_fig_bool is True:
    plt.savefig(f'Analysis_{current_dataset_name}_loss_history__{plot_legend_strings}_{train_start_time}.png')
plt.show()

plot_different_parameter_train_accuracy_history = convert_to_list(data_train_accuracy_history_total)
plot_different_parameter_test_accuracy_history = convert_to_list(data_test_accuracy_history_total)
if plot_show_train_bool is True:
    for i, plot_train_accuracy_history in enumerate(plot_different_parameter_train_accuracy_history):
        plt.plot(plot_train_accuracy_history, label=f"Train Accuracy History with {plot_legend_strings} = {plot_parameters_list[i]}")
if plot_show_test_bool is True:
    for i, plot_test_accuracy_history in enumerate(plot_different_parameter_test_accuracy_history):
        plt.plot(plot_test_accuracy_history, label=f"Test Accuracy History with {plot_legend_strings} = {plot_parameters_list[i]}")
plt.xlabel("Epochs")
if plot_log_scale is True:
    plt.ylabel("Log Accuracy")
    plt.yscale('log')
else:
    plt.ylabel("Accuracy")
plt.title("Accuracy History" + plot_title_strings)
plt.legend()
if plot_save_fig_bool is True:
    plt.savefig(f'Analysis_{current_dataset_name}_accuracy_history__{plot_legend_strings}_{train_start_time}.png')
plt.show()

plot_different_parameter_train_error_history = convert_to_list(data_train_error_history_total)
plot_different_parameter_test_error_history = convert_to_list(data_test_error_history_total)
if plot_show_train_bool is True:
    for i, plot_train_error_history in enumerate(plot_different_parameter_train_error_history):
        plt.plot(plot_train_error_history, label=f"Train Error History with {plot_legend_strings} = {plot_parameters_list[i]}")
if plot_show_test_bool is True:
    for i, plot_test_error_history in enumerate(plot_different_parameter_test_error_history):
        plt.plot(plot_test_error_history, label=f"Test Error History with {plot_legend_strings} = {plot_parameters_list[i]}")
plt.xlabel("Epochs")
if plot_log_scale is True:
    plt.ylabel("Log Error")
    plt.yscale('log')
else:
    plt.ylabel("Error")
plt.title("Error History" + plot_title_strings)
plt.legend()
if plot_save_fig_bool is True:
    plt.savefig(f'Analysis_{current_dataset_name}_error_history__{plot_legend_strings}_{train_start_time}.png')
plt.show()

**Plot for Batchsize-Accuracy curve.**

In [ ]:
plot_batchsize_list = [1]
plot_save_fig_bool = False
plot_show_train_bool = True
plot_show_test_bool = False
plot_log_scale = False

plot_different_parameter_train_accuracy_history = convert_to_list(data_train_accuracy_history_total)
plot_different_parameter_test_accuracy_history = convert_to_list(data_test_accuracy_history_total)
plot_batchsize_accuracy_point_train = []
plot_batchsize_accuracy_point_test = []
if plot_show_train_bool is True:
    for plot_train_accuracy_history in plot_different_parameter_train_accuracy_history:
        plot_batchsize_accuracy_point_train.append(plot_train_accuracy_history[-1])
    plt.plot(plot_batchsize_list, plot_batchsize_accuracy_point_train, marker='o', linestyle='-', label=f"Batchsize vs Train Accuracy")
if plot_show_test_bool is True:
    for plot_test_accuracy_history in plot_different_parameter_test_accuracy_history:
        plot_batchsize_accuracy_point_test.append(plot_test_accuracy_history[-1])
    plt.plot(plot_batchsize_list, plot_batchsize_accuracy_point_test, marker='x', linestyle='-', label=f"Batchsize vs Test Accuracy")
plt.xlabel("Batchsize")
if plot_log_scale is True:
    plt.ylabel("Log Accuracy")
    plt.yscale('log')
else:
    plt.ylabel("Accuracy")
plt.title("Batchsize-Accuracy History")
plt.legend()
if plot_save_fig_bool is True:
    plt.savefig(f'Analysis_{current_dataset_name}_batchsize_accuracy_{train_start_time}.png')
plt.show()

**Plot for Batchsize-Runtime curve.**

In [ ]:
plot_batchsize_list = [1]
plot_save_fig_bool = False
plot_log_scale = False

plot_different_parameter_time_history = convert_to_list(data_time_history_total)
plot_batchsize_runtime_point = []
for plot_time_history in plot_different_parameter_time_history:
    plot_batchsize_runtime_point.append(plot_time_history[-1])
plt.plot(plot_batchsize_list, plot_batchsize_runtime_point, marker='o', linestyle='-', label=f"Batchsize vs Runtime")
plt.xlabel("Batchsize")
if plot_log_scale is True:
    plt.ylabel("Log Runtime")
    plt.yscale('log')
else:
    plt.ylabel("Runtime")
plt.title("Batchsize-Runtime History")
plt.legend()
if plot_save_fig_bool is True:
    plt.savefig(f'Analysis_{current_dataset_name}_batchsize_runtime_{train_start_time}.png')
plt.show()